# The Gladius Data Model

### Data Model Structure


Collection: "clubs"

Document: "club_id"
Club data fields
Sub-collection: "events"
Document: "event_id"
Event data fields
Sub-collection: "occurrences"
Document: "occurrence_id"
Occurrence data fields
Sub-collection: "attendance"
Document: "attendance_id"
Field: "user_id" (to reference the attending user)
Field: "score" (record the score for the user at this occurrence)

Collection: "users"

Document: "user_id"
User data fields
Sub-collection: "scores"
Document: "score_id"
Field: "club_id" (to reference the club)
Field: "event_id" (to reference the event)
Field: "occurrence_id" (to reference the occurrence)
Field: "score" (the user's score for this event occurrence)


In [37]:
# Installs
#!pip install firebase-admin

In [ ]:

# Import firebase_admin and initialize the app
import firebase_admin
from firebase_admin import auth, credentials, firestore
import random


Authentication

In [ ]:
if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Create users with authentication

### Delete all auth users

In [27]:
user_list = auth.list_users()

# Iterate over each user and delete them
for user in user_list.users:
    auth.delete_user(user.uid)
    print(f"Deleted user: {user.uid}")

print("All users have been deleted.")

Deleted user: 7DtR60Mjc7WH3phO6LGSkK3buei2
Deleted user: HyU4ZLPLPYUH8ir8ukl4qSwVrqO2
Deleted user: IfXh2ortOCZmr8q7WuZujxwrQFj1
Deleted user: JZtbBhFRmChzoqlIeUu9lPLWwHO2
Deleted user: K5PvdXF6A8ZqFfze37owXIIoYKO2
Deleted user: ROJY5RdB78dfl9QQrrriBpUl1cX2
Deleted user: SJcM2b9fwQZfpFGOaiLYX6IQwQj2
Deleted user: X0wR2xjZ5xN6RZEG88dwyE1LUZh2
Deleted user: aYqGWD5vJPRx9yuJx6IX92664ZJ2
Deleted user: bjeTJCEg6Uhv7iuvWa10qBNDPVb2
Deleted user: yqFab2OtYeP7uwBh8Ocl010bv132
All users have been deleted.


### Create new auth users

In [28]:
# Create a list of users to add
users = ["Mare", "Jüri", "Liisa", "Mart", "Evelin", "Tõnis", "Mari", "Henrik", "Triin", "Jaak", 'Bob']


user_uuids = {}
# Loop through the users and create them with a default password
for user in users:
  try:
    # Create the user with the email and password
    user_record = auth.create_user(
      email=f'{user.lower()}@example.com',
      password='123456'
    )
    user_uuids[user] = user_record.uid
    # Print the user ID
    print(f'Successfully created new user: {user_record.uid}')
  except firebase_admin.auth.AuthError as e:
    # Handle any errors
    print(f'Error creating new user: {e}')

Successfully created new user: QOOc8hZLVrP0965Uh6pSSVd24Jf2
Successfully created new user: NeUlhJOHrYf9wJ4pUqAUhOxft7E2
Successfully created new user: BDUFoR5dM0eIsNQ4tDe9ixEQGgR2
Successfully created new user: 5Vu5WYh8ypRNE77xvzNr4IPYQwS2
Successfully created new user: Z7SPsuiRCyewNItseSRx2BbxAah1
Successfully created new user: BLpUXWzidWezz2XpFCAABUbzvpL2
Successfully created new user: efNgErA6oxZAZS2IcV2riYHQlJL2
Successfully created new user: HvA1ZUXS5Rcekb9QiuC2oyAgx7E3
Successfully created new user: 63Ni3tgHZNQNpZMJPJCnaUXgBUI2
Successfully created new user: PlOydoMjIch5K6rVUszQmlKwIG83
Successfully created new user: SOMzKHrZ5ThGIKUsGjofNzYcLIE2


### Delete user collection

In [35]:
## Delete user collection
# Function to delete a collection and its documents
def delete_collection(collection_ref, batch_size):
    docs = collection_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(collection_ref, batch_size)

# Specify the "users" collection to be deleted
users_collection_ref = db.collection("users")

# Delete the "users" collection and its documents
delete_collection(users_collection_ref, 500)

print("Users collection deleted successfully.")

Users collection deleted successfully.


## Add users collection

In [29]:
# Create a collection for users
users_collection = db.collection("users")

user_roles = {}

# Loop through the users and add them to the "users" collection
for user in users:
    age = random.randint(18, 60)
    email = f"{user.lower()}@example.com"
    occupation = random.choice(["Student", "Engineer", "Teacher", "Artist"])


for user in users:

    # Assign roles for each user in different clubs
    if user in ["Bob", "Mare"]:
        user_roles[user_uuids[user]] = {"club_id": "1", "role": "owner"}
    else:
        user_roles[user_uuids[user]] = {"club_id": "1", "role": "athlete"}

    user_data = {
        "name": user,
        "email": f"{user.lower()}@example.com",
        "age": random.randint(18, 60),
        "occupation": random.choice(["Student", "Engineer", "Teacher", "Artist"]),
        "is_active": random.choice([True, False]),
    }

    user_ref = users_collection.document(user_uuids[user])
    user_ref.set(user_data)
    print(f"Successfully created user: {user_uuids[user]}")

Successfully created user: QOOc8hZLVrP0965Uh6pSSVd24Jf2
Successfully created user: NeUlhJOHrYf9wJ4pUqAUhOxft7E2
Successfully created user: BDUFoR5dM0eIsNQ4tDe9ixEQGgR2
Successfully created user: 5Vu5WYh8ypRNE77xvzNr4IPYQwS2
Successfully created user: Z7SPsuiRCyewNItseSRx2BbxAah1
Successfully created user: BLpUXWzidWezz2XpFCAABUbzvpL2
Successfully created user: efNgErA6oxZAZS2IcV2riYHQlJL2
Successfully created user: HvA1ZUXS5Rcekb9QiuC2oyAgx7E3
Successfully created user: 63Ni3tgHZNQNpZMJPJCnaUXgBUI2
Successfully created user: PlOydoMjIch5K6rVUszQmlKwIG83
Successfully created user: SOMzKHrZ5ThGIKUsGjofNzYcLIE2


## Delete Club collection

In [36]:
## Delete user collection
# Function to delete a collection and its documents
def delete_collection(collection_ref, batch_size):
    docs = collection_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(collection_ref, batch_size)

# Specify the "users" collection to be deleted
users_collection_ref = db.collection("clubs")

# Delete the "users" collection and its documents
delete_collection(users_collection_ref, 500)

print("Clubs collection deleted successfully.")

Clubs collection deleted successfully.


## Add club collection

In [33]:

# Create a collection for clubs
clubs = db.collection("clubs")

# Create a document for a club with some data fields
club_id = '1'
club_data = {
    "name": "Gladius Club",
    "description": "A Learn to Earn Club",
    "owners": [user_uuid for user_uuid, role in user_roles.items() if role["club_id"] == "1" and role["role"] == "owner"],
    "athletes": [user_uuid for user_uuid, role in user_roles.items() if role["club_id"] == "1"],

}
club_ref = clubs.document(club_id)
club_ref.set(club_data)


update_time {
  seconds: 1687370976
  nanos: 378830000
}

### TODO: Add events to clubs

In [ ]:
# Create a sub-collection for events under the club document
events = club_ref.collection("events")

# Create a document for an event with some data fields
event_id = "456"
event_data = {
    "title": "Fencing Tournament",
    "date": "2023-06-30",
    "location": "Gladius Arena"
}
event_ref = events.document(event_id)
event_ref.set(event_data)

# Create a sub-collection for occurrences under the event document
occurrences = event_ref.collection("occurrences")

# Create a document for an occurrence with some data fields
occurrence_id = "789"
occurrence_data = {
    "start_time": "14:00",
    "end_time": "16:00",
    "participants": ["Alice", "Bob"]
}
occurrence_ref = occurrences.document(occurrence_id)
occurrence_ref.set(occurrence_data)

# Create a sub-collection for attendance under the occurrence document
attendance = occurrence_ref.collection("attendance")

# Create a document for an attendance with some data fields
attendance_id = "1011"
attendance_data = {
    "name": "Alice",
    "score": 3
}
attendance_ref = attendance.document(attendance_id)
attendance_ref.set(attendance_data)

update_time {
  seconds: 1687370617
  nanos: 317941000
}